In [1]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/evivancovid/warehouse"). \
    enableHiveSupport(). \
    appName(f'evivancovid | Python - Data Processing - Overview'). \
    master('yarn'). \
    getOrCreate()

In [10]:
from pyspark.sql.functions import current_date, current_timestamp, lit, to_date, to_timestamp

l = [("X", )]
df = spark.createDataFrame(l).toDF("dummy")
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



# Date and Time common manipulations

In [7]:
df.select(current_date()).show() #yyyy-MM-dd <---- Standard Date Format 

+--------------+
|current_date()|
+--------------+
|    2022-05-13|
+--------------+



In [8]:
df.select(current_timestamp()).show(truncate=False) #yyyy-MM-dd HH:mm:ss.SSS <---- Standard TimeStamp Format 

+-----------------------+
|current_timestamp()    |
+-----------------------+
|2022-05-13 01:12:31.539|
+-----------------------+



In [9]:
# We can convert a string which contain date or timestamp in non-standard format to standard date or time using to_date or to_timestamp function respectively.

In [11]:
#Important: "yyyyMMdd" is teh format inw hich pyspark will return the date

df.select(to_date(lit('20210228'), 'yyyyMMdd').alias('to_date')).show()

+----------+
|   to_date|
+----------+
|2021-02-28|
+----------+



In [12]:
df.select(to_timestamp(lit('20210228 1725'), 'yyyyMMdd HHmm').alias('to_timestamp')).show()

+-------------------+
|       to_timestamp|
+-------------------+
|2021-02-28 17:25:00|
+-------------------+



# Date and Time Arithmetic

In [24]:
from pyspark.sql.functions import datediff, months_between, add_months, round,date_add, date_sub, col

In [5]:
# Create a DataFrame from this list of tuples
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]

In [6]:
#Columns shouyld be of type STRING in order to perform arithmetic and manipulations
datetimesDF = spark.createDataFrame(datetimes, schema = "date STRING, timestamp STRING")

In [7]:
#Testing everything is in order
datetimesDF.show()

+----------+--------------------+
|      date|           timestamp|
+----------+--------------------+
|2014-02-28|2014-02-28 10:00:...|
|2016-02-29|2016-02-29 08:08:...|
|2017-10-31|2017-12-31 11:59:...|
|2019-11-30|2019-08-31 00:00:...|
+----------+--------------------+



In [9]:
#Add 10 days to both date and time values.
#Subtract 10 days from both date and time values.

datetimesDF.select("date","timestamp"). \
withColumn("date_add_10", date_add("date", 10)). \
withColumn("time_add_10", date_add("timestamp", 10)). \
withColumn("day_sub_10", date_sub("date", 10)). \
withColumn("time_sub_10", date_sub("timestamp", 10)). \
show()

+----------+--------------------+-----------+-----------+----------+-----------+
|      date|           timestamp|date_add_10|time_add_10|day_sub_10|time_sub_10|
+----------+--------------------+-----------+-----------+----------+-----------+
|2014-02-28|2014-02-28 10:00:...| 2014-03-10| 2014-03-10|2014-02-18| 2014-02-18|
|2016-02-29|2016-02-29 08:08:...| 2016-03-10| 2016-03-10|2016-02-19| 2016-02-19|
|2017-10-31|2017-12-31 11:59:...| 2017-11-10| 2018-01-10|2017-10-21| 2017-12-21|
|2019-11-30|2019-08-31 00:00:...| 2019-12-10| 2019-09-10|2019-11-20| 2019-08-21|
+----------+--------------------+-----------+-----------+----------+-----------+



In [11]:
help(datediff)

Help on function datediff in module pyspark.sql.functions:

datediff(end, start)
    Returns the number of days from `start` to `end`.
    
    .. versionadded:: 1.5.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08','2015-05-10')], ['d1', 'd2'])
    >>> df.select(datediff(df.d2, df.d1).alias('diff')).collect()
    [Row(diff=32)]



In [16]:
#Get the difference between current_date and date values as well as current_timestamp and time values.

datetimesDF.select("date", "timestamp"). \
withColumn("currentDate", current_date()). \
withColumn("date_diff", datediff(current_date(), "date")). \
withColumn("time_diff", datediff(current_date(), "timestamp")). \
show(truncate = False)

+----------+-----------------------+-----------+---------+---------+
|date      |timestamp              |currentDate|date_diff|time_diff|
+----------+-----------------------+-----------+---------+---------+
|2014-02-28|2014-02-28 10:00:00.123|2022-05-13 |2996     |2996     |
|2016-02-29|2016-02-29 08:08:08.999|2022-05-13 |2265     |2265     |
|2017-10-31|2017-12-31 11:59:59.123|2022-05-13 |1655     |1594     |
|2019-11-30|2019-08-31 00:00:00.000|2022-05-13 |895      |986      |
+----------+-----------------------+-----------+---------+---------+



In [12]:
help(current_date)

Help on function current_date in module pyspark.sql.functions:

current_date()
    Returns the current date at the start of query evaluation as a :class:`DateType` column.
    All calls of current_date within the same query return the same value.
    
    .. versionadded:: 1.5



In [28]:
#Get the number of months between current_date and date values as well as current_timestamp and time values.
#Add 3 months to both date values as well as time values.

datetimesDF.select("date", "timestamp"). \
withColumn("currentDate", current_date()). \
withColumn("date_between", months_between(current_date(), "date")). \
withColumn("time_between", months_between(current_timestamp(), "timestamp")). \
withColumn("date_3months", add_months("date", 3)). \
withColumn("time_3months", add_months("timestamp", 3)). \
show(truncate = False)

+----------+-----------------------+-----------+------------+------------+------------+------------+
|date      |timestamp              |currentDate|date_between|time_between|date_3months|time_3months|
+----------+-----------------------+-----------+------------+------------+------------+------------+
|2014-02-28|2014-02-28 10:00:00.123|2022-05-13 |98.51612903 |98.50510827 |2014-05-28  |2014-05-28  |
|2016-02-29|2016-02-29 08:08:08.999|2022-05-13 |74.48387097 |74.47535618 |2016-05-29  |2016-05-29  |
|2017-10-31|2017-12-31 11:59:59.123|2022-05-13 |54.41935484 |52.40564628 |2018-01-31  |2018-03-31  |
|2019-11-30|2019-08-31 00:00:00.000|2022-05-13 |29.4516129  |32.42177494 |2020-02-29  |2019-11-30  |
+----------+-----------------------+-----------+------------+------------+------------+------------+



In [17]:
help(add_months)

Help on function add_months in module pyspark.sql.functions:

add_months(start, months)
    Returns the date that is `months` months after `start`
    
    .. versionadded:: 1.5.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(add_months(df.dt, 1).alias('next_month')).collect()
    [Row(next_month=datetime.date(2015, 5, 8))]



In [22]:
help(months_between)

Help on function months_between in module pyspark.sql.functions:

months_between(date1, date2, roundOff=True)
    Returns number of months between dates date1 and date2.
    If date1 is later than date2, then the result is positive.
    If date1 and date2 are on the same day of month, or both are the last day of month,
    returns an integer (time of day will be ignored).
    The result is rounded off to 8 digits unless `roundOff` is set to `False`.
    
    .. versionadded:: 1.5.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00', '1996-10-30')], ['date1', 'date2'])
    >>> df.select(months_between(df.date1, df.date2).alias('months')).collect()
    [Row(months=3.94959677)]
    >>> df.select(months_between(df.date1, df.date2, False).alias('months')).collect()
    [Row(months=3.9495967741935485)]



# Date and Time Trunc Functions

In [2]:
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]

In [4]:
datetimesDF = spark.createDataFrame(datetimes, schema = "date STRING, time STRING")

In [6]:
datetimesDF.show(truncate = False)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+



In [11]:
# Get beginning month date using date field and beginning year date using time field.

from pyspark.sql.functions import trunc

datetimesDF. \
withColumn("date_trunc", trunc("date", "MM")). \
withColumn("time_trunc", trunc("time", "yy")). \
show(truncate = False)

+----------+-----------------------+----------+----------+
|date      |time                   |date_trunc|time_trunc|
+----------+-----------------------+----------+----------+
|2014-02-28|2014-02-28 10:00:00.123|2014-02-01|2014-01-01|
|2016-02-29|2016-02-29 08:08:08.999|2016-02-01|2016-01-01|
|2017-10-31|2017-12-31 11:59:59.123|2017-10-01|2017-01-01|
|2019-11-30|2019-08-31 00:00:00.000|2019-11-01|2019-01-01|
+----------+-----------------------+----------+----------+



#### Difference between above and below example is **trunc()** truncates only to the date, **date_trunc()** truncates time as well.

In [12]:
# Get beginning hour time using date and time field

from pyspark.sql.functions import date_trunc

datetimesDF. \
withColumn("date_trunc", date_trunc('MM', "date")). \
withColumn("time_trunc", date_trunc('yy', "time")). \
show(truncate=False)

+----------+-----------------------+-------------------+-------------------+
|date      |time                   |date_trunc         |time_trunc         |
+----------+-----------------------+-------------------+-------------------+
|2014-02-28|2014-02-28 10:00:00.123|2014-02-01 00:00:00|2014-01-01 00:00:00|
|2016-02-29|2016-02-29 08:08:08.999|2016-02-01 00:00:00|2016-01-01 00:00:00|
|2017-10-31|2017-12-31 11:59:59.123|2017-10-01 00:00:00|2017-01-01 00:00:00|
|2019-11-30|2019-08-31 00:00:00.000|2019-11-01 00:00:00|2019-01-01 00:00:00|
+----------+-----------------------+-------------------+-------------------+



# Date and Time Extract Functions

Here are the common extract functions: year, month, weekofyear, dayofyear, dayofmonth, dayofweek, hour, minute, second. They are self explanatory.

In [6]:
from pyspark.sql.functions import year, month, weekofyear, dayofmonth, \
    dayofyear, dayofweek, current_date

In [4]:
l = [("X", )]
df = spark.createDataFrame(l).toDF("dummy")
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



The following functions only work with **DATES** 

In [21]:
df.select(
    current_date().alias('current_date'), 
    year(current_date()).alias('year'),
    month(current_date()).alias('month'),
    weekofyear(current_date()).alias('weekofyear'),
    dayofyear(current_date()).alias('dayofyear'),
    dayofmonth(current_date()).alias('dayofmonth'),
    dayofweek(current_date()).alias('dayofweek')
).show(truncate = False) #yyyy-MM-dd

+------------+----+-----+----------+---------+----------+---------+
|current_date|year|month|weekofyear|dayofyear|dayofmonth|dayofweek|
+------------+----+-----+----------+---------+----------+---------+
|2022-05-13  |2022|5    |19        |133      |13        |6        |
+------------+----+-----+----------+---------+----------+---------+



In [7]:
from pyspark.sql.functions import current_timestamp, hour, minute, second

The following functions work with **DATES** and **TIMESTAMPS**

In [8]:
df.select(
    current_timestamp().alias('current_timestamp'), 
    year(current_timestamp()).alias('year'),
    month(current_timestamp()).alias('month'),
    dayofmonth(current_timestamp()).alias('dayofmonth'),
    hour(current_timestamp()).alias('hour'),
    minute(current_timestamp()).alias('minute'),
    second(current_timestamp()).alias('second')
).show(truncate=False) #yyyy-MM-dd HH:mm:ss.SSS

+-----------------------+----+-----+----------+----+------+------+
|current_timestamp      |year|month|dayofmonth|hour|minute|second|
+-----------------------+----+-----+----------+----+------+------+
|2022-05-13 15:25:41.254|2022|5    |13        |15  |25    |41    |
+-----------------------+----+-----+----------+----+------+------+



# Using to_date and to_timestamp

We can use *to_date()* and *to_timestamp()* to convert non standard dates and timestamps to standard formats.
1. **yyyyMMdd** is the standard date format
2. **dd-MMM-yyyy HH:mm:ss.SSS** is the standard timestamp format


In [2]:
datetimes = [(20140228, "28-Feb-2014 10:00:00.123"),
                     (20160229, "20-Feb-2016 08:08:08.999"),
                     (20171031, "31-Dec-2017 11:59:59.123"),
                     (20191130, "31-Aug-2019 00:00:00.000")
                ]

In [3]:
datetimesDF = spark.createDataFrame(datetimes, schema = "date STRING, time STRING")

In [4]:
datetimesDF.show(truncate = False)

+--------+------------------------+
|date    |time                    |
+--------+------------------------+
|20140228|28-Feb-2014 10:00:00.123|
|20160229|20-Feb-2016 08:08:08.999|
|20171031|31-Dec-2017 11:59:59.123|
|20191130|31-Aug-2019 00:00:00.000|
+--------+------------------------+



In [5]:
from pyspark.sql.functions import lit, to_date, to_timestamp, col

In [6]:
help(to_date)

Help on function to_date in module pyspark.sql.functions:

to_date(col, format=None)
    Converts a :class:`~pyspark.sql.Column` into :class:`pyspark.sql.types.DateType`
    using the optionally specified format. Specify formats according to `datetime pattern`_.
    By default, it follows casting rules to :class:`pyspark.sql.types.DateType` if the format
    is omitted. Equivalent to ``col.cast("date")``.
    
    .. _datetime pattern: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
    
    .. versionadded:: 2.2.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
    >>> df.select(to_date(df.t).alias('date')).collect()
    [Row(date=datetime.date(1997, 2, 28))]
    
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
    >>> df.select(to_date(df.t, 'yyyy-MM-dd HH:mm:ss').alias('date')).collect()
    [Row(date=datetime.date(1997, 2, 28))]



In [7]:
help(to_timestamp)

Help on function to_timestamp in module pyspark.sql.functions:

to_timestamp(col, format=None)
    Converts a :class:`~pyspark.sql.Column` into :class:`pyspark.sql.types.TimestampType`
    using the optionally specified format. Specify formats according to `datetime pattern`_.
    By default, it follows casting rules to :class:`pyspark.sql.types.TimestampType` if the format
    is omitted. Equivalent to ``col.cast("timestamp")``.
    
    .. _datetime pattern: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
    
    .. versionadded:: 2.2.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
    >>> df.select(to_timestamp(df.t).alias('dt')).collect()
    [Row(dt=datetime.datetime(1997, 2, 28, 10, 30))]
    
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
    >>> df.select(to_timestamp(df.t, 'yyyy-MM-dd HH:mm:ss').alias('dt')).collect()
    [Row(dt=datetime.datetime(1997, 2, 28, 10, 30))]



In [11]:
# Convert data in datetimesDF to standard dates or timestamps

datetimesDF.show()

+--------+--------------------+
|    date|                time|
+--------+--------------------+
|20140228|28-Feb-2014 10:00...|
|20160229|20-Feb-2016 08:08...|
|20171031|31-Dec-2017 11:59...|
|20191130|31-Aug-2019 00:00...|
+--------+--------------------+



In [20]:
datetimesDF. \
    withColumn("to_date", to_date(col("date"), "yyyyMMdd")). \
    withColumn("to_timestamp", to_timestamp(col("time"), "dd-MMM-yyyy HH:mm:ss.SSS")). \
show(truncate = False)

+--------+------------------------+----------+-----------------------+
|date    |time                    |to_date   |to_timestamp           |
+--------+------------------------+----------+-----------------------+
|20140228|28-Feb-2014 10:00:00.123|2014-02-28|2014-02-28 10:00:00.123|
|20160229|20-Feb-2016 08:08:08.999|2016-02-29|2016-02-20 08:08:08.999|
|20171031|31-Dec-2017 11:59:59.123|2017-10-31|2017-12-31 11:59:59.123|
|20191130|31-Aug-2019 00:00:00.000|2019-11-30|2019-08-31 00:00:00    |
+--------+------------------------+----------+-----------------------+



# Using date_format() Function

We can use *date_format* to extract the required information in a desired format from standard date or timestamp

In [22]:
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]

datetimesDF = spark.createDataFrame(datetimes, schema="date STRING, time STRING")
datetimesDF.show(truncate=False)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+



In [23]:
# Get the year and month from both date and time columns using yyyyMM format.
# Also make sure that the data type is converted to integer.

In [29]:
from pyspark.sql.functions import date_format

In [25]:
help(date_format)

Help on function date_format in module pyspark.sql.functions:

date_format(date, format)
    Converts a date/timestamp/string to a value of string in the format specified by the date
    format given by the second argument.
    
    A pattern could be for instance `dd.MM.yyyy` and could return a string like '18.03.1993'. All
    pattern letters of `datetime pattern`_. can be used.
    
    .. _datetime pattern: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
    
    .. versionadded:: 1.5.0
    
    Notes
    -----
    Whenever possible, use specialized functions like `year`.
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_format('dt', 'MM/dd/yyy').alias('date')).collect()
    [Row(date='04/08/2015')]



In [35]:
datetimesDF. \
    withColumn("date_ym", date_format("date", "yyyyMM").cast("int")). \
    withColumn("time_ym", date_format("time", "yyyyMM").cast("int")). \
show(truncate = False)

+----------+-----------------------+-------+-------+
|date      |time                   |date_ym|time_ym|
+----------+-----------------------+-------+-------+
|2014-02-28|2014-02-28 10:00:00.123|201402 |201402 |
|2016-02-29|2016-02-29 08:08:08.999|201602 |201602 |
|2017-10-31|2017-12-31 11:59:59.123|201710 |201712 |
|2019-11-30|2019-08-31 00:00:00.000|201911 |201908 |
+----------+-----------------------+-------+-------+



In [38]:
#Get the information from time in yyyyMMddHHmmss format.

datetimesDF. \
    withColumn("date_ym", date_format("date", "yyyyMM").cast("int")). \
    withColumn("time_ym", date_format("time", "yyyyMMddHHmmss").cast("long")). \
show(truncate = False)

+----------+-----------------------+-------+--------------+
|date      |time                   |date_ym|time_ym       |
+----------+-----------------------+-------+--------------+
|2014-02-28|2014-02-28 10:00:00.123|201402 |20140228100000|
|2016-02-29|2016-02-29 08:08:08.999|201602 |20160229080808|
|2017-10-31|2017-12-31 11:59:59.123|201710 |20171231115959|
|2019-11-30|2019-08-31 00:00:00.000|201911 |20190831000000|
+----------+-----------------------+-------+--------------+



In [41]:
#Get year and day of year uisng yyyyDDD format

datetimesDF. \
    withColumn("date_yd", date_format("date", "yyyyDDD").cast("int")). \
    withColumn("time_yd", date_format("time", "yyyyDDD").cast("int")). \
show(truncate = False)

+----------+-----------------------+-------+-------+
|date      |time                   |date_yd|time_yd|
+----------+-----------------------+-------+-------+
|2014-02-28|2014-02-28 10:00:00.123|2014059|2014059|
|2016-02-29|2016-02-29 08:08:08.999|2016060|2016060|
|2017-10-31|2017-12-31 11:59:59.123|2017304|2017365|
|2019-11-30|2019-08-31 00:00:00.000|2019334|2019243|
+----------+-----------------------+-------+-------+



In [42]:
#get complete description of the date

datetimesDF. \
    withColumn("date_description", date_format("date", "MMMM dd, yyyy")). \
show(truncate = False)

+----------+-----------------------+-----------------+
|date      |time                   |date_description |
+----------+-----------------------+-----------------+
|2014-02-28|2014-02-28 10:00:00.123|February 28, 2014|
|2016-02-29|2016-02-29 08:08:08.999|February 29, 2016|
|2017-10-31|2017-12-31 11:59:59.123|October 31, 2017 |
|2019-11-30|2019-08-31 00:00:00.000|November 30, 2019|
+----------+-----------------------+-----------------+



In [44]:
# Get name of the week day using date.
# EE = Abbreviated name, EEEE = Full name

datetimesDF. \
    withColumn("date_abbr", date_format("date", "EE")). \
    withColumn("date_full", date_format("date", "EEEE")). \
show(truncate = False)

+----------+-----------------------+---------+---------+
|date      |time                   |date_abbr|date_full|
+----------+-----------------------+---------+---------+
|2014-02-28|2014-02-28 10:00:00.123|Fri      |Friday   |
|2016-02-29|2016-02-29 08:08:08.999|Mon      |Monday   |
|2017-10-31|2017-12-31 11:59:59.123|Tue      |Tuesday  |
|2019-11-30|2019-08-31 00:00:00.000|Sat      |Saturday |
+----------+-----------------------+---------+---------+



# Dealing with Unix Timestamp

Unix Timestamp is an integer that started from January 1st 1970 Midnight UTC, also known as epoch, and is incremented by 1 every second.

We can convert regular date/timestamp to Unix Timestamp using **unix_timestamp**.

We can convert to regular date/timestamp from Unix using **from_unixtime**

In [46]:
datetimes = [(20140228, "2014-02-28", "2014-02-28 10:00:00.123"),
                     (20160229, "2016-02-29", "2016-02-29 08:08:08.999"),
                     (20171031, "2017-10-31", "2017-12-31 11:59:59.123"),
                     (20191130, "2019-11-30", "2019-08-31 00:00:00.000")
                ]

datetimesDF = spark.createDataFrame(datetimes).toDF("dateid", "date", "time")
datetimesDF.show(truncate=False)

+--------+----------+-----------------------+
|dateid  |date      |time                   |
+--------+----------+-----------------------+
|20140228|2014-02-28|2014-02-28 10:00:00.123|
|20160229|2016-02-29|2016-02-29 08:08:08.999|
|20171031|2017-10-31|2017-12-31 11:59:59.123|
|20191130|2019-11-30|2019-08-31 00:00:00.000|
+--------+----------+-----------------------+



In [47]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, col